In [1]:
# https://www.youtube.com/watch?v=8gPONnGIPgw&ab_channel=Murtaza%27sWorkshop-RoboticsandAI

In [2]:
import cv2
from cvzone.HandTrackingModule import HandDetector as htm
# from handtrack import HandDetector as htm
import numpy as np
import time
import autopy

In [3]:
wCam, hCam = 640, 480
wScr, hScr = autopy.screen.size()
plocx, plocy = 0, 0
clocx, clocy = 0, 0
frameR = 100 # frame reduction
stn = 10 # smoothening
cap = cv2.VideoCapture(0)
cap.set(3, wCam) #weight 
cap.set(4, hCam) #hight
pTime = 0
detector = htm(detectionCon=0.5, maxHands=2)

while True:
    # find hand landmarks
    success, img = cap.read()
    img = cv2.flip(img, 1)
    hands, img = detector.findHands(img)
    if hands :
        # Hand 1
        hand1 = hands[0]
        lmList = hand1["lmList"]  # List of 21 Landmark points
        bbox1 = hand1["bbox"]  # Bounding box info x,y,w,h
        centerPoint1 = hand1['center']  # center of the hand cx,cy
        handType1 = hand1["type"]  # Handtype Left or Right

        
    
        # get the tip of the index and middle hands
        if len(lmList) != 0:
            x1, y1 = lmList[8][0:2]
            x2, y2 = lmList[12][0:2]
#             cv2.circle(img, (x1,y1), 15, (0,255,255), cv2.FILLED)
#             print(x1, y1, x2, y2)
            
            # check which fingers are up
            fingers = detector.fingersUp(hand1)
#             print(fingers)
            
            # only index fingers : move mode
            if fingers[1]==1 and fingers[2]==0:

                # convert coordinates
                cv2.rectangle(img, (frameR,frameR), (wCam-frameR,hCam-frameR), (0,0,255), 2)
                x3 = np.interp(x1, (frameR,wCam-frameR), (0,wScr))
                y3 = np.interp(y1, (frameR,hCam-frameR), (0,hScr))
                
                # smoothen values
                clocx = plocx + (x3 - plocx) / stn
                clocy = plocy + (y3 - plocy) / stn
                plocx, plocy = clocx, clocy
                
                # move mouse
                autopy.mouse.move(clocx, clocy)
                cv2.circle(img, (x1,y1), 15, (0,255,255), cv2.FILLED)

#             # both index and middle fingers are up and find distance between fingers
            if fingers[1]==1 and fingers[2]==1:
                length, info, img = detector.findDistance((x1,y1),(x2,y2),img)
#                 print(length)

            # click mouse if distance short
                if length <= 20:
                    cv2.circle(img, (info[4], info[5]), 15, (0,255,0), cv2.FILLED)
                    autopy.mouse.click()
        
        # frame rate
        cTime = time.time()
        fps = 1/(cTime - pTime)
        pTime = cTime
        cv2.putText(img, str(int(fps)), (20,50), cv2.FONT_HERSHEY_PLAIN, 3, (255,0,0), 3)

       #display
        cv2.imshow("Image", img)
        if (cv2.waitKey(1) & 0xff == ord('e')):
            break    
    else :
        # frame rate
        cTime = time.time()
        fps = 1/(cTime - pTime)
        pTime = cTime
        cv2.putText(img, str(int(fps)), (20,50), cv2.FONT_HERSHEY_PLAIN, 3, (255,0,0), 3)

       #display
        cv2.imshow("Image", img)
        if (cv2.waitKey(1) & 0xff == ord('e')):
            break
            
cap.release()
cv2.destroyAllWindows()